# Data Exploration

Here we perform some preliminary data analyses and formulate our questions and hypotheses at the end.

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

db_path = 'postgres:///music'
engine = create_engine(db_path)

table_names = engine.table_names()

table_names

['reviews', 'artists', 'genres', 'labels', 'years', 'content']

In [2]:
def postgres_to_df(table):
    engine = create_engine(db_path)
    conn = engine.connect()
    result = conn.execute(f'SELECT * FROM {table}')
    columns = [x for x in result.keys()]
    df = pd.DataFrame(result.fetchall(),columns=columns)
    return df

In [3]:
postgres_to_df('reviews').head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017


# Is there a statistically meaningful difference between the scores of the jazz and metal genres? Two tailed test
$H_o$: Songs of the Jazz Genre will not have a different rating allocation from Metal

$H_a$:

$\alpha$:
# Will the label with the most artists have the highests scores? One tailed test
$H_o$: Will the label with the most artists have higher scores than the label with the least artists signed?

$H_a$:

$\alpha$:
# Is there a relationship between labels scores and genres? One tailed test
$H_o$: The label with the most genre diversity will not have higher scores than the label with least genre diversity

$H_a$:

$\alpha$:
# Have average song ratings changed from 2000-2009 to 2010-2017?
$H_o$: Average song ratings have stayed the same year over year

$H_a$:

$\alpha$:
